In [1]:
import os
import numpy as np
from astropy.table import Table, vstack



In [3]:
import os
import numpy as np
from astropy.table import Table, vstack, Column

# Fields and filters
fields = [f"{i:02d}" for i in range(33)]
filters = ['f115wn', 'f150wn', 'f200wn']
ap_indices = [f'{i}' for i in range(1, 7)]

# Output table list
all_tables = []

for field in fields:
    filename = f'sex-{field}_phot_apcorr.fits'
    if not os.path.exists(filename):
        print(f"Skipping missing file: {filename}")
        continue

    cat = Table.read(filename)

    # Base columns to retain
    base_cols = ['id', 'ra', 'dec']
    data = {col: cat[col] for col in base_cols}

    # Add field name as a column
    data['field'] = Column([f'sex-{field}'] * len(cat), dtype='str')

    # Loop through each filter and aperture index
    for filt in filters:
        for i in ap_indices:
            flux_col = f'{filt}_tot_{i}'
            err_col = f'{filt}_etot_{i}'

            data[flux_col] = cat[flux_col] if flux_col in cat.colnames else np.full(len(cat), -99.0)
            data[err_col] = cat[err_col] if err_col in cat.colnames else np.full(len(cat), -99.0)

    # Create table for this field and append
    tab = Table(data)
    all_tables.append(tab)

# Combine all field tables
combined = vstack(all_tables, metadata_conflicts='silent')

# Save the result
combined.write('combined_sex-00-32_phot_apcorr_catalog.fits', overwrite=True)
print("Saved to combined_phot_apcorr_catalog.fits")


Saved to combined_phot_apcorr_catalog.fits
